In [2]:
import os; os.environ["DISPLAY"] = "localhost:10.0"; 
os.environ["__GLX_VENDOR_LIBRARY_NAME"] = "mesa"; # export __GLX_VENDOR_LIBRARY_NAME=mesa   See: https://superuser.com/questions/106056/force-software-based-opengl-rendering-on-ubuntu
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3";
import open3d as o3d
import numpy as np
import cv2

from trimesh_render import lookAt

import pytransform3d.transformations as pt

class PointCloudViewer:
    def __init__(self):
        self.origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
        self.eef = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
        self.eef_T_inv = np.eye(4)
        self.pcd = o3d.geometry.PointCloud()

        # Initialize the pointcloud viewer
        self.vis = o3d.visualization.Visualizer()
        self.vis.create_window(window_name="Point Cloud")

        self.vis.add_geometry(self.origin)
        self.vis.add_geometry(self.eef)
        self.vis.add_geometry(self.pcd)

        self.vis.get_render_option().point_size = 1
        # self.vis.get_render_option().background_color = np.asarray([0, 0, 0])

        view_control = self.vis.get_view_control()
        view_control.set_constant_z_far(1000)

        # Retrieve the camera parameters
        camera_params = view_control.convert_to_pinhole_camera_parameters()
        # Set the extrinsic parameters, yz_flip is for Open3D camera configuration
        # camera_pose = lookAt(eye=np.array([0., 0., -1.]), target=np.array([0. ,0., 0.]), up=np.array([0.0, -1.0, 0.0]), yz_flip=True)
        camera_pose = lookAt(eye=np.array([0., -1., -1.]), target=np.array([0. ,0., 0.]), up=np.array([0.0, -1.0, 0.0]), yz_flip=True)
        camera_params.extrinsic = np.linalg.inv(camera_pose)
        # Set the camera parameters
        view_control.convert_from_pinhole_camera_parameters(camera_params)
        
        frame = self.vis.capture_screen_float_buffer(do_render=True)
        frame = np.asarray(frame)
        self.video_writer = cv2.VideoWriter("demo.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame.shape[1], frame.shape[0]))
    
    def update_vis(self):
        # Update the visualizer
        self.vis.poll_events()
        self.vis.update_renderer()
    
    def update(self, image_array, depth_array, intrinsics, Teef2cam):
        im_rgb = o3d.geometry.Image(cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB))
        im_d = o3d.geometry.Image(depth_array)

        im_rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
            im_rgb,
            im_d,
            depth_scale=1000.0,
            convert_rgb_to_intensity=False
        )

        height, width, _ = image_array.shape

        new_pcd = o3d.geometry.PointCloud.create_from_rgbd_image(im_rgbd, o3d.camera.PinholeCameraIntrinsic(width=width, height=height, intrinsic_matrix=intrinsics))

        # new_pcd.transform(CAMERA_TO_WORLD)

        self.pcd.points = new_pcd.points
        self.pcd.colors = new_pcd.colors
        self.vis.update_geometry(self.pcd)

        self.eef.transform(self.eef_T_inv)
        self.eef.transform(Teef2cam)
        self.eef_T_inv = pt.invert_transform(Teef2cam)
        self.vis.update_geometry(self.eef)

        self.update_vis()
        
        # Capture the current frame
        frame = self.vis.capture_screen_float_buffer(do_render=True)
        frame = np.asarray(frame)
        
        # Convert to uint8 and BGR format for OpenCV
        frame = (frame * 255).astype(np.uint8)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        # Write frame to video
        self.video_writer.write(frame)
    
    def close(self):
        self.video_writer.release()
        self.vis.destroy_window()

viewer = PointCloudViewer()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


In [3]:
import json

droid3d_anno_path = "droid"

with open(droid3d_anno_path + "/cam2base_extrinsics.json") as f:
    cam2base_extrinsics = json.load(f)

with open(droid3d_anno_path + "/cam2base_extrinsic_superset.json") as f:
    cam2base_extrinsic_superset = json.load(f)

with open(droid3d_anno_path + "/droid_language_annotations.json") as f:
    droid_language_annotations = json.load(f)

print(len(cam2base_extrinsic_superset))

24044


In [4]:
from utils import xyz_from_uvd, uvd_from_xyz, T_from_xyzrpy, xyzrpy_from_T

import glob
import json

droid_base_path = "/mnt/20T/droid_raw_1.0.1"

episode_meta_paths = glob.glob("*/success/*/*/metadata_*.json", root_dir=droid_base_path)
print(len(episode_meta_paths))

def wo_prefix(p):
    return "/".join(p.split("/")[3:])

for episode_meta_path in episode_meta_paths:
    with open(droid_base_path + "/" + episode_meta_path, "r") as f:
        metadata = json.load(f)
        
    uuid = metadata["uuid"]
    
    if uuid not in cam2base_extrinsic_superset:
        print(f"uuid {uuid} not in cam2base_extrinsic_superset")
        continue
    
    prefix_path = "/".join(episode_meta_path.split("/")[:-1])
    
    print(uuid)
    print(prefix_path)
    print(metadata)

    break

59740
uuid IPRL+7790ec0a+2023-06-27-20h-57m-09s not in cam2base_extrinsic_superset
uuid IPRL+7790ec0a+2023-06-27-20h-24m-51s not in cam2base_extrinsic_superset
IPRL+7790ec0a+2023-06-27-21h-02m-21s
IPRL/success/2023-06-27/Tue_Jun_27_21:02:21_2023
{'uuid': 'IPRL+7790ec0a+2023-06-27-21h-02m-21s', 'lab': 'IPRL', 'user': 'Marion Lepert', 'user_id': '7790ec0a', 'date': '2023-06-27', 'timestamp': '2023-06-27-21h-02m-21s', 'hdf5_path': 'success/2023-06-27/Tue_Jun_27_21:02:21_2023/trajectory.h5', 'building': '775 Roble', 'scene_id': 8351259745, 'success': True, 'robot_serial': 'panda-295341-1325480', 'r2d2_version': '1.3', 'current_task': 'Open or close hinged object (ex: hinged door, microwave, oven, book, dryer, toilet, box)', 'trajectory_length': 153, 'wrist_cam_serial': '12391924', 'ext1_cam_serial': '21582473', 'ext2_cam_serial': '28221883', 'wrist_cam_extrinsics': [0.32085092844058694, 0.061907514889166, 0.4160269019638068, 2.780198025634674, -0.21272213182917477, 1.7303129617793613], 'ex

In [5]:
import pytransform3d.transformations as pt
import numpy as np

Tbase2cam = pt.invert_transform(T_from_xyzrpy(cam2base_extrinsic_superset[uuid][metadata["ext1_cam_serial"]]))

Teef2tip = np.array([
    [0, 0, 1, 0],
    [0, -1, 0, 0],
    [1, 0, 0, 0.150],
    [0, 0, 0, 1]
])

In [6]:
import h5py

cartesian_positions = None

with h5py.File(droid_base_path + "/" + prefix_path + "/" + wo_prefix(metadata["hdf5_path"]), "r") as f:
    cartesian_positions = np.array(f["observation"]["robot_state"]["cartesian_position"])

In [7]:
# f = h5py.File(droid_base_path + "/" + prefix_path + "/" + wo_prefix(metadata["hdf5_path"]), "r")

In [8]:
# f['action'].keys()

In [9]:
import cv2
import numpy as np
import pyzed.sl as sl

def load_svo(svo_path, frame_idx=None):
    # Create a ZED camera object
    zed = sl.Camera()
    
    # Set configuration parameters
    init_params = sl.InitParameters()
    init_params.set_from_svo_file(svo_path)
    
    # Open the camera
    err = zed.open(init_params)
    if err != sl.ERROR_CODE.SUCCESS:
        print(f"Error {err}: Failed to open SVO file")
        return
    
    # Get intrinsics
    left_cam_calibration_params = zed.get_camera_information().camera_configuration.calibration_parameters.left_cam
    fx = left_cam_calibration_params.fx
    fy = left_cam_calibration_params.fy
    cx = left_cam_calibration_params.cx
    cy = left_cam_calibration_params.cy
    intrinsics = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    depth_scale = 1000
    
    # Create image
    image = sl.Mat()
    depth = sl.Mat()
    point_cloud = sl.Mat()
    
    # Read frames
    i = 0
    while zed.grab() == sl.ERROR_CODE.SUCCESS:
        zed.retrieve_image(image, sl.VIEW.LEFT) # Retrieve image
        zed.retrieve_measure(depth, sl.MEASURE.DEPTH) # Retrieve depth
        zed.retrieve_measure(point_cloud, sl.MEASURE.XYZRGBA)
        
        # zed.retrieve_image(image_r, sl.VIEW.RIGHT) # Retrieve right image
        # zed.retrieve_measure(depth_r, sl.MEASURE.DEPTH_RIGHT) # Retrieve right depth
        # zed.retrieve_measure(point_cloud_r, sl.MEASURE.XYZRGBA_RIGHT)

        # Convert to numpy array
        image_array = image.get_data().copy() # 不加copy好像会导致内存泄漏
        depth_array = depth.get_data().copy()
        
        a = yield image_array, depth_array, intrinsics, depth_scale
        
        if frame_idx is not None and i == frame_idx:
            break
        
        i += 1
    
    # Close the camera
    zed.close()
    
    return

In [11]:
from utils_cv import draw_xyz_axis

g = load_svo(droid_base_path + "/" + prefix_path + "/" + wo_prefix(metadata["ext1_svo_path"]))

for idx, (image_array, depth_array, intrinsics, depth_scale) in enumerate(g):
    image_array = cv2.cvtColor(np.asarray(image_array), cv2.COLOR_BGRA2BGR)
    depth_array = depth_array.astype(np.uint16)
    
    Ttip2base = T_from_xyzrpy(cartesian_positions[idx])
    Ttip2cam = Tbase2cam @ Ttip2base
    Teef2cam = Ttip2cam @ Teef2tip
    
    viewer.update(image_array, depth_array, intrinsics, Teef2cam)
    
    image_array_with_axis = draw_xyz_axis(image_array, ob_in_cam=Teef2cam, scale=0.1, K=intrinsics, thickness=3, transparency=0)
    
    # cv2.imshow("image", image_array)
    depth_colored = cv2.applyColorMap(cv2.convertScaleAbs(depth_array / depth_scale, alpha=100), cv2.COLORMAP_JET)
    # cv2.imshow("depth_colored", depth_colored)
    cv2.imshow("image", np.vstack((cv2.resize(image_array_with_axis, (0, 0), fx=0.5, fy=0.5), cv2.resize(depth_colored, (0, 0), fx=0.5, fy=0.5))))
    key = cv2.waitKey(1)
    if key == 27:
        break

viewer.close()

[2025-06-18 17:19:39 UTC][ZED][INFO] Logging level INFO
[2025-06-18 17:19:39 UTC][ZED][INFO] Logging level INFO
[2025-06-18 17:19:39 UTC][ZED][INFO] Logging level INFO
[2025-06-18 17:19:39 UTC][ZED][INFO] [Init]  Depth mode: PERFORMANCE
[2025-06-18 17:19:39 UTC][ZED][INFO] [Init]  Serial Number: S/N 21582473
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


/tmp/ipykernel_2860268/3423563367.py:7: RuntimeWarning: invalid value encountered in cast
  depth_array = depth_array.astype(np.uint16)


[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] GLFW Err

KeyboardInterrupt: 